In [1]:
import numpy as np
import math
import pandas as pd

In [11]:
# randomly generate x,y
x = np.random.rand(2)
y = (x[0]**2 + x[1]**2)/2

# first hidden layer
w1 = np.random.rand(10,2)
b1 = np.random.rand(10,1)

# second hidden layer
w2 = np.random.rand(10,10)
b2 = np.random.rand(10,1)

#output layer
w3 = np.random.rand(10)
b3 = np.random.rand(10)

# Gradient calculation with script

In [4]:
def sigmoid(x):
  # sigmoid function
  return 1 / (1 + math. exp(-x))

def forward(x,w,b,act,layer,torch_flag = False):
  if layer == 1:
    v1 = []
    z = x * w + b
    ze = 0
    for i in range(10):
      for j in range(2):
        ze += z[i][j]
        vv = act(ze)
      v1.append(vv)
    if torch_flag:
      return v1,z
    # the format of data type should be change when use pytorch
    return np.array(v1),z 
  
  # second layer pass
  if layer == 2:
    z = x * w + b
    v2 = []
    ze = 0
    for i in range(10):
      for j in range(10):
        ze += z[i][j]
      vv = act(ze)
      v2.append(vv)
    if torch_flag:
      return v2,z
    return np.array(v2),z 
    
  # third layer pass
  if layer == 3:
    if torch_flag:
        zz = 0
        for i in range(10):
          zz += x[i] * w[i] + b[i]
        return zz
    z = x * w + b
    return np.sum(z)

def loss(pred,y):
  # loss function
  L = (pred- y)**2
  return L

In [5]:
# dL/dw3 = 2(yred-y) * v2
def gradient_w3(v,y,pred):
  dw3 = []
  for i in range(10):
    dw = 2 *v[i]* (pred - y)
    dw3.append(dw)
  return np.array(dw3)

# dL/db3 = 2(yred-y)
def gradient_b3(y,pred):
  db3 = []
  for i in range(10):
    db = 2 * (pred - y)
    db3.append(db)
  return np.array(db3)

In [6]:
#dL/dw2 = 2(yred-y) * w3*sig(z2) * (1-sig(z2)) * v1
def gradient_w2(v,y,pred,z2,w3):
  dw2 = []
  ze = 0
  sigz2 = []
  for i in range(10):
    for j in range(10):
        ze += z2[i][j]
    sigz = sigmoid(ze) * (1 - sigmoid(ze))
    # save the sig(z2), no need to calculate next time
    sigz2.append(sigz)
    dw = 2 *v[i]* (pred - y) * sigz *w3[i]
    dw2.append(dw)
  return np.array(dw2),np.array(sigz2)
#dL/db2 = 2(yred-y) * w3*sig(z2) * (1-sig(z2))
def gradient_b2(y,pred,sigz2,w3):
  db2 = []
  for i in range(10):
    db = 2 * (pred - y)* sigz2[i] *w3[i]
    db2.append([db])
  return np.array(db2)

In [7]:
# dL/dw1 = 2 * (yred - y)* w3 *sig(z2) * w2 * sig(z1) * x
def gradient_w1(x,y,w2,w3,z1,pred,sigz2):
  dw1 = []
  sigz1 = []
  for i in range(10):
    sigz = sigmoid(z1[i][0] + z1[i][1]) * (1 - sigmoid(z1[i][0] + z1[i][1]))
    sigz1.append(sigz)
    w2sum = np.sum(w2[i])
    dwtemp1 = 2 * x[0] * (pred - y) * w3[i] * sigz2[i] * w2sum * sigz
    dwtemp2 = 2 * x[1] * (pred - y) * w3[i] * sigz2[i] * w2sum * sigz
    dw1.append([dwtemp1,dwtemp2])
  return np.array(dw1),np.array(sigz1)

# dL/db1 = 2 * (yred - y)* w3 *sig(z2) * w2 * sig(z1)
def gradient_b1(y,pred,sigz2,sigz1,w2):
  db1 = []
  for i in range(10):
    w2sum = np.sum(w2[i])
    db = 2 * (pred - y)*w3[i]*sigz2[i] * w2sum * sigz1[i]
    db1.append([db])
  return np.array(db1)

In [17]:
# forward 
v1,z1 = forward(x,w1,b1,sigmoid,1)
v2,z2 = forward(v1,w2,b2,sigmoid,2)
y_pred = forward(v2,w3,b3,sigmoid,3)
print('The predicted y is: '+ str(round(y_pred,2)))

# loss
L = loss(y_pred,y)

# gradient decent
dw3 = gradient_w3(v2,y,y_pred)
db3 = gradient_b3(y,y_pred)
dw2,sigz2 = gradient_w2(v1,y,y_pred,z2,w3)
db2 = gradient_b2(y,y_pred,sigz2,w3)
dw1, sigz1 = gradient_w1(x,y,w2,w3,z1,y_pred,sigz2)
db1 = gradient_b1(y,y_pred,sigz2,sigz1,w2)


print('-------------dw3------------')
print(dw3)
print('-------------db3------------')
print(db3)
print('-------------dw2------------')
print(dw2)
print('-------------db2------------')
print(db2)
print('-------------dw1------------')
print(dw1)
print('-------------db1------------')
print(db1)


db2 = np.reshape(db2,10)
db1 = np.reshape(db1,10)
dw1 = np.reshape(dw1,20)
dw11 = dw1[:10]
dw12 = dw1[10:]

output_df = pd.DataFrame({'dw3': dw3,'db3':db3,'dw2': dw2,'db2':db2,'dw11': dw11,'dw12':dw12,'db1':db1}) 
output_df.to_csv('gradient decent script.dat')


The predicted y is: 8.36
-------------dw3------------
[15.58998543 15.59228346 15.59228351 15.59228351 15.59228351 15.59228351
 15.59228351 15.59228351 15.59228351 15.59228351]
-------------db3------------
[15.59228351 15.59228351 15.59228351 15.59228351 15.59228351 15.59228351
 15.59228351 15.59228351 15.59228351 15.59228351]
-------------dw2------------
[8.45728103e-04 4.84804552e-08 5.46632872e-16 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
-------------db2------------
[[9.15226727e-04]
 [4.88109015e-08]
 [5.47253208e-16]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
-------------dw1------------
[[2.55289484e-04 1.69807009e-04]
 [1.50851951e-08 1.00339890e-08]
 [3.47921055e-16 2.31421337e-16]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0

# Deap learning process with script

In [9]:
learning_rate = 0.01
for epoch in range(10):
  # forward 
  v1,z1 = forward(x,w1,b1,sigmoid,1)
  v2,z2 = forward(v1,w2,b2,sigmoid,2)
  y_pred = forward(v2,w3,b3,sigmoid,3)

  # loss
  L = loss(y_pred,y)

  # gradient decent
  dw3 = gradient_w3(v2,y,y_pred)
  db3 = gradient_b3(y,y_pred)
  dw2,sigz2 = gradient_w2(v1,y,y_pred,z2,w3)
  db2 = gradient_b2(y,y_pred,sigz2,w3)
  dw1, sigz1 = gradient_w1(x,y,w2,w3,z1,y_pred,sigz2)
  db1 = gradient_b1(y,y_pred,sigz2,sigz1,w2)

  # update w and b 
  w3 -= learning_rate * dw3
  b3 -= learning_rate * db3
  w2 -= learning_rate * dw2
  b2 -= learning_rate * db2
  w1 -= learning_rate * dw1
  b1 -= learning_rate * db1

  if epoch % 1 == 0:
    print(f'epoch{epoch + 1}: loss = {L}')

epoch1: loss = 142.64456701911698
epoch2: loss = 51.3603192337142
epoch3: loss = 18.492701088767955
epoch4: loss = 6.658448327141117
epoch5: loss = 2.397428867029006
epoch6: loss = 0.8632139069983521
epoch7: loss = 0.31080723920078707
epoch8: loss = 0.11190869243280575
epoch9: loss = 0.04029364129859838
epoch10: loss = 0.014508055548621534


# Gradience calculation with pytorch

In [10]:
pip install torch

In [13]:
def forward(x,w,b,act,layer,torch_flag = False):
  if layer == 1:
    v1 = []
    z = x * w + b
    ze = 0
    for i in range(10):
      for j in range(2):
        ze += z[i][j]
        vv = act(ze)
      v1.append(vv)
    if torch_flag:
      return v1,z
    # the format of data type should be change when use pytorch
    return np.array(v1),z 
  
  # second layer pass
  if layer == 2:
    z = x * w + b
    v2 = []
    ze = 0
    for i in range(10):
      for j in range(10):
        ze += z[i][j]
      vv = act(ze)
      v2.append(vv)
    if torch_flag:
      return v2,z
    return np.array(v2),z 
    
  # third layer pass
  if layer == 3:
    if torch_flag:
        zz = 0
        for i in range(10):
          zz += x[i] * w[i] + b[i]
        return zz
    z = x * w + b
    return np.sum(z)

In [14]:
# calculate gradient decent with torch
import torch
x_train = torch.tensor(x,requires_grad=True)
y_train = torch.tensor(y,requires_grad=True)

tw1 = torch.tensor(w1, requires_grad=True)
tb1 = torch.tensor(b1, requires_grad=True)
tw2 = torch.tensor(w2, requires_grad=True)
tb2 = torch.tensor(b2, requires_grad=True)
tw3 = torch.tensor(w3, requires_grad=True)
tb3 = torch.tensor(b3, requires_grad=True)

# active function
m = torch.nn.Sigmoid()
# forward 
v1,z = forward(x_train,tw1,tb1,m,1,True)
tv1 = torch.stack(v1)
v2,z = forward(tv1,tw2,tb2,m,2,True)
tpred = forward(v2,tw3,tb3,m,3,True)

# loss
L = loss(tpred,y_train)
# backward
L.backward()

print('The predicted y is:')
print(tpred)
print('-------------dw3------------')
print(tw3.grad)
print('-------------db3------------')
print(tb3.grad)
print('-------------dw2------------')
print(tw2.grad)
print('-------------db2------------')
print(tb2.grad)
print('-------------dw1------------')
print(tw1.grad)
print('-------------db1------------')
print(tb1.grad)

torch_df = pd.DataFrame({'dw3': tw3.grad,'db3':tb3.grad,'dw2': tw2.grad,
                         'db2':tb2.grad,'dw1': tw1.grad,'db1':tb1.grad,}) 
torch_df.to_csv('gradient decent torch.dat')


The predicted y is:
tensor(8.3631, dtype=torch.float64, grad_fn=<AddBackward0>)
-------------dw3------------
tensor([15.5900, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923,
        15.5923, 15.5923], dtype=torch.float64)
-------------db3------------
tensor([15.5923, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923, 15.5923,
        15.5923, 15.5923], dtype=torch.float64)
-------------dw2------------
tensor([[8.4577e-04, 9.0908e-04, 9.1424e-04, 9.1515e-04, 9.1526e-04, 9.1527e-04,
         9.1528e-04, 9.1528e-04, 9.1528e-04, 9.1528e-04],
        [4.5104e-08, 4.8480e-08, 4.8756e-08, 4.8804e-08, 4.8810e-08, 4.8811e-08,
         4.8811e-08, 4.8811e-08, 4.8811e-08, 4.8811e-08],
        [5.0570e-16, 5.4355e-16, 5.4663e-16, 5.4718e-16, 5.4725e-16, 5.4725e-16,
         5.4725e-16, 5.4725e-16, 5.4725e-16, 5.4725e-16],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 

# Deep learning process with pytorch

In [ ]:
import torch

torch.manual_seed(0)
x_train = torch.rand(1,2)
y_train = torch.empty(1,1)
y_train[0,0] = (x_train[0][0]**2 + x_train[0][1]**2)/2


class DLmodel(torch.nn.Module):
    # build the class the this 2 hidden layer deep learning model
    def __init__(self):
        # initiate the model with corresponding input and output
        super(DLmodel, self).__init__()
        self.linear1 = torch.nn.Linear(2,10)
        self.linear2 = torch.nn.Linear(10,10)
        self.linear3 = torch.nn.Linear(10,1)

    def forward(self, x):
        # forward pass
        sigmoid = torch.nn.Sigmoid()
        h1_sigmoid = sigmoid(self.linear1(x_train))
        h2_sigmoid = sigmoid(self.linear2(h1_sigmoid))   
        y_pred = self.linear3(h2_sigmoid)
        return y_pred

# construct model
model = DLmodel()

# loss function
loss_fn = torch.nn.MSELoss(reduction='sum')

# optimising process
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
for t in range(15):
  # forward pass
  y_pred = model(x_train)
  # loss
  loss = loss_fn(y_pred, y_train)
  print('step %d: loss=%f   ' % (t, loss.item()))   
  # zero gradientssuch that the gradient could not be accumulated
  optimizer.zero_grad()
  # perform a backward pass
  loss.backward()
  #update the weights
  optimizer.step()

'''This is how you can print the weight and bias of each layer, but not know how to save it properly'''
print(y)
print(y_pred)
print(model.linear3.bias.grad)
print(model.linear3.weight.grad)
print(model.linear2.bias.grad)
print(model.linear2.weight.grad)
print(model.linear1.bias.grad)
print(model.linear1.weight.grad)

step 0: loss=0.986679   
step 1: loss=0.064272   
step 2: loss=0.004507   
step 3: loss=0.000315   
step 4: loss=0.000022   
step 5: loss=0.000002   
step 6: loss=0.000000   
step 7: loss=0.000000   
step 8: loss=0.000000   
step 9: loss=0.000000   
step 10: loss=0.000000   
step 11: loss=0.000000   
step 12: loss=0.000000   
step 13: loss=0.000000   
step 14: loss=0.000000   
tensor([-1.1921e-07])
tensor([[-4.1819e-08, -5.9634e-08, -6.5727e-08, -6.3175e-08, -7.6438e-08,
         -5.6411e-08, -6.1303e-08, -5.3890e-08, -7.1631e-08, -4.9985e-08]])
tensor([-1.0607e-08, -3.3048e-09,  3.9322e-09,  1.9669e-10, -1.0532e-08,
        -3.6911e-09,  5.3326e-10,  3.5020e-09,  3.7761e-09,  4.4017e-09])
tensor([[-3.2148e-09, -6.6929e-09, -6.1228e-09, -5.8128e-09, -6.4215e-09,
         -2.1870e-09, -3.8417e-09, -6.0604e-09, -3.3973e-09, -8.1288e-09],
        [-1.0016e-09, -2.0852e-09, -1.9076e-09, -1.8110e-09, -2.0007e-09,
         -6.8137e-10, -1.1969e-09, -1.8882e-09, -1.0585e-09, -2.5326e-09],
   